

Hybrid with CF: final score = 0.7*embed_cos + 0.3*CF_sim

## Assemble a DS

In [2]:
import pandas as pd

art = pd.read_csv(
    "../../data/processed/articles_clean.csv",
    usecols=["sku","groupId", "name", "brand", "description", "audience", "category", "priceSEK", "color", "fabrics"],
    dtype=str
)

In [3]:
# Remove articles with priceSEK below 1 from the DataFrame and convert to numeric
art['priceSEK'] = pd.to_numeric(art['priceSEK'], errors='coerce')
art = art[art['priceSEK'] >= 1]

In [4]:
# Create price buckets based on the distribution of price_sek, using 6 buckets
price_bins = [0, 100, 300, 600, 1000, 2000, float('inf')]
price_labels = [
    'Budget',        # 0-100
    'Value',         # 100-300
    'Popular',       # 300-600
    'Premium',       # 600-1000
    'Luxury',        # 1000-2000
    'Exclusive'      # 2000+
]
art['priceband'] = pd.cut(art['priceSEK'], bins=price_bins, labels=price_labels, include_lowest=True)
art['priceband'] = art['priceband'].astype(object)
art.loc[art['priceSEK'].isna(), 'priceband'] = 'unknown'

In [5]:
with pd.option_context('display.max_columns', None, 'display.width', 0):
    display(art.sort_values("groupId"))

,sku,groupId,name,description,brand,color,audience,category,priceSEK,fabrics,priceband
11326,055522,055522,Tröja,Sticka en färgglad och trendig tröja i garnet ...,Gjestal Garn,NaN,dam,Tröjor,29.0,NaN,Budget
11325,055573,055573,Luva,Sticka en trendig huva i garnet Halaus från No...,Novita,vit,dam,"Mössor & hattar,Mönster",29.0,NaN,Budget
11324,055575,055575,Vantar,Sticka ett par vantar med blomstermotiv i garn...,Novita,vit,dam,Vantar,29.0,NaN,Budget
11323,055576,055576,Benvärmare,Sticka ett par trendiga benvärmare i garnet Ha...,Novita,vit,dam,Sockor & strumpor,29.0,NaN,Budget
11310,095318,095302,Garn Drops Nepal,Garn Drops Nepal är ett underbart tjockt luxuö...,Drops Design,svart,unknown,unknown,33.0,"['alpaca', 'ull']",Budget
...,...,...,...,...,...,...,...,...,...,...,...
9,AH2031-4547,AH2031,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,svart-grå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
14,AH2031-4244,AH2031,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,svart-grå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
15,AH3021-4244,AH3021,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,blå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
16,AH3021-4547,AH3021,Stödstrumpa Herr,FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organi...,Funq Wear,blå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value


In [6]:
# Deduplicate so that for each groupId, keep the first row for all columns except 'color', 
# which should be a list of all colors from merged rows (excluding missing/unknown/nan/none).
def merge_colors(series):
    # Remove missing/unknown/nan/none and deduplicate
    colors = [str(c).strip() for c in series if pd.notna(c) and str(c).strip().lower() not in {"", "unknown", "nan", "none"}]
    return list(sorted(set(colors))) if colors else []

art = art.sort_values("sku")  # Ensure deterministic "first" row
art = art.groupby("groupId", as_index=False).agg(
    {col: (merge_colors if col == "color" else "first") for col in art.columns if col != "sku"}
)


## 1. Build a clean text field for vectorization

In [7]:
import pandas as pd, numpy as np, unicodedata, re, ast

# -----------------------------
# Constants & tiny helpers
# -----------------------------
MISSING = {"", "unknown", "nan", "none", None}

BAD_CATS = {
    # marketing / temporal / state tags that shouldn't drive semantic similarity
    "REA", "Nyheter", "Kampanj", "Outlet", "Erbjudande",
    "Sale", "Black Friday", "Jul", "Sommar"
}

def canon(s: str) -> str:
    """
    Normalize Unicode, standardize dashes/spaces, trim.
    Keep original casing (model is cased).
    """
    s = unicodedata.normalize("NFKC", str(s))
    s = re.sub(r"\u00A0", " ", s)                       # nbsp
    s = re.sub(r"[\u2010-\u2015\u2212\-]+", "-", s)     # dashes
    s = re.sub(r"\s+", " ", s).strip()
    return s

def as_list(x):
    """
    Convert value to list[str] without transforming tokens.
    Keeps colors/materials exactly as provided if already lists.
    """
    if isinstance(x, (list, tuple, np.ndarray, pd.Series)):
        return [str(v) for v in x if str(v).strip() not in MISSING]
    s = str(x).strip()
    if s in MISSING:
        return []
    if s.startswith("[") and s.endswith("]"):
        try:
            vals = ast.literal_eval(s)
            return [str(v) for v in vals if str(v).strip() not in MISSING]
        except Exception:
            return [s]
    return [s]

def norm_categories(x):
    """
    Parse comma-separated categories -> canonicalized list (dedup, order-preserving).
    """
    cats = [canon(c) for c in str(x).split(",") if str(c).strip() not in MISSING]
    seen_lower, out = set(), []
    for c in cats:
        cl = c.lower()
        if c and cl not in seen_lower:
            seen_lower.add(cl)
            out.append(c)
    return out

def primary_categories(cats, max_keep=2):
    """
    Keep up to max_keep *semantic* categories (drop marketing/state tags like REA).
    """
    cats = cats or []
    good = [c for c in cats if c not in BAD_CATS]
    return good[:max_keep]

def short_desc(desc, max_words=30):
    """
    First sentence of description, truncated by word count.
    """
    if not desc:
        return ""
    # split at sentence boundary; then trim words
    first = re.split(r"(?<=[.!?])\s+", desc)[0]
    toks = first.split()
    return " ".join(toks[:max_words])

# -----------------------------
# Build fields on a copy of your source df `art`
# -----------------------------
art = art.copy()

# Colors & materials: keep EXACTLY as they are (no lowercasing/splitting/mapping)
art["colors_norm"]    = art["color"].apply(as_list)
art["materials_norm"] = art["fabrics"].apply(as_list)

# Categories: keep full list for metadata, plus a semantic subset for embeddings
art["categories_norm_full"] = art["category"].apply(norm_categories)
art["categories_semantic"]  = art["categories_norm_full"].apply(lambda cs: primary_categories(cs, max_keep=2))

# Optional: a REA flag (useful for filtering/UX; has zero effect on embeddings)
art["sale_tag"] = art["categories_norm_full"].apply(lambda cs: "REA" in set(cs))

# -----------------------------
# Natural text for embeddings (no boilerplate labels)
# - Include: name, short desc, brand, semantic categories, materials
# - Exclude: colors, price, price band, audience, marketing tags (like REA)
# -----------------------------
def build_text_embed_clean(r):
    name  = canon(r.get("name", ""))
    desc0 = canon(r.get("description", ""))
    desc  = short_desc(desc0, max_words=30)

    brand_raw = r.get("brand", "")
    brand = canon(brand_raw) if str(brand_raw).lower() not in MISSING else ""

    cats = r.get("categories_semantic", []) or []
    mats = r.get("materials_norm", []) or []

    parts = []
    if name:
        parts.append(f"{name}.")
    if desc:
        parts.append(desc)

    # Add salient attributes as plain tokens (no labels like 'Varumärke:' / 'Kategori:')
    attrs = []
    if brand:
        attrs.append(brand)
    if cats:
        attrs.append(", ".join(cats))
    if mats:
        attrs.append(", ".join(mats))

    if attrs:
        parts.append(" ".join(attrs) + ".")

    txt = " ".join(parts)
    return re.sub(r"\s+", " ", txt).strip()

art["text"] = art.apply(build_text_embed_clean, axis=1)

# -----------------------------
# Final table for retrieval
# -----------------------------
group_df = art[[
    "groupId",
    "text",
    "colors_norm",          # kept as-is for filtering or hybrid rerank
    "materials_norm",       # kept as-is; also lightly included in text
    "categories_norm_full", # full list incl. REA etc. (metadata/UI)
    "categories_semantic",  # semantic subset used in text
    "brand",
    "priceband"
]].reset_index(drop=True)

# This is the corpus you embed with your Swedish SBERT (no prefixes)
corpus = group_df["text"].tolist()


In [8]:
pd.set_option('display.max_colwidth', None)
group_df.head()

,groupId,text,colors_norm,materials_norm,categories_norm_full,categories_semantic,brand,priceband
0,055522,Tröja. Sticka en färgglad och trendig tröja i garnet Vera från House of Yarn! Gjestal Garn Tröjor None.,[],[None],[Tröjor],[Tröjor],Gjestal Garn,Budget
1,055573,"Luva. Sticka en trendig huva i garnet Halaus från Novita! Novita Mössor & hattar, Mönster None.",[vit],[None],"[Mössor & hattar, Mönster]","[Mössor & hattar, Mönster]",Novita,Budget
2,055575,Vantar. Sticka ett par vantar med blomstermotiv i garnet Halaus från Novita! Novita Vantar None.,[vit],[None],[Vantar],[Vantar],Novita,Budget
3,055576,Benvärmare. Sticka ett par trendiga benvärmare i garnet Halaus från Novita! Novita Sockor & strumpor None.,[vit],[None],[Sockor & strumpor],[Sockor & strumpor],Novita,Budget
4,095302,"Garn Drops Nepal. Garn Drops Nepal är ett underbart tjockt luxuöst garn spunnet av 35% superfin alpaca och 65 % peruvian highland ull. Drops Design alpaca, ull.","[beige, blå, blå-grön, brun, grå, gråmelerad, grön, gul, lila, marin, mörkgrå, orange, rosa, röd, svart, turkos, vit]","[alpaca, ull]",[],[],Drops Design,Budget


In [9]:
import sys, torch, transformers, sentence_transformers
print("py exe:", sys.executable)
print("torch:", torch.__version__, "at", torch.__file__)
print("transformers:", transformers.__version__)
print("sentence-transformers:", sentence_transformers.__version__)


py exe: /opt/conda/bin/python
torch: 2.8.0+cpu at /opt/conda/lib/python3.10/site-packages/torch/__init__.py
transformers: 4.56.1
sentence-transformers: 5.1.0


In [ ]:

#python -m pip install --upgrade --index-url https://download.pytorch.org/whl/cpu torch==2.8.0

#pip install --no-deps sentence-transformers transformers tokenizers huggingface_hub safetensors

#pip install faiss-cpu scikit-learn

#pip install "huggingface_hub[hf_xet]"  # or: pip install hf_xet


#Embed

from sentence_transformers import SentenceTransformer
import faiss, numpy as np, pandas as pd

#Recommended RAG pipeline: Hybrid retrieval (use BGE-M3’s dense+sparse) → re-ranker (e.g., bge-reranker / v2). Works well with Vespa and Milvus. https://huggingface.co/BAAI/bge-m3
enc = SentenceTransformer("BAAI/bge-m3", device="cpu") # three retrieval modes (dense + sparse + multi-vector/ColBERT, 100+ languages, and long context (≤8192 tokens).
enc.max_seq_length = 256
corpus = group_df["text"].tolist()
E = enc.encode(corpus, batch_size=64, normalize_embeddings=True).astype("float32")


KeyboardInterrupt: 

In [ ]:
N, d = E.shape

# ---------- FAISS (cosine via IP on normalized vectors) ----------
index = faiss.IndexFlatIP(d)
index.add(E)

In [ ]:

# ----- Online helpers ----- run FAISS index.search on the fly, return top-k now (fast, per request).
def _filter_topk(I_row, D_row, k, mask=None, drop_idx=None):
    out_idx, out_sim = [], []
    for j, s in zip(I_row, D_row):
        if drop_idx is not None and j == drop_idx:
            continue
        if mask is None or mask[j]:
            out_idx.append(j); out_sim.append(float(s))
            if len(out_idx) == k:
                break
    return out_idx, out_sim

def search_by_gid(gid, k=200, require_color=None):
    i = group_df.index[group_df.groupId.eq(gid)][0]
    D, I = index.search(E[i:i+1], k + 50)  # cushion then filter
    mask = None
    if require_color:
        mask = group_df.colors_norm.apply(lambda cs: require_color in cs).to_numpy()
    I0, D0 = _filter_topk(I[0], D[0], k, mask=mask, drop_idx=i)
    res = group_df.iloc[I0].copy()
    res["similarity"] = D0
    return res

def search_by_text(q, k=200, require_color=None):
    # Symmetric text query (no query:/passage: prompts)
    qv = enc.encode([q], normalize_embeddings=True).astype("float32")
    D, I = index.search(qv, k + 50)
    mask = None
    if require_color:
        mask = group_df.colors_norm.apply(lambda cs: require_color in cs).to_numpy()
    I0, D0 = _filter_topk(I[0], D[0], k, mask=mask)
    res = group_df.iloc[I0].copy()
    res["similarity"] = D0
    return res


In [ ]:
# ----- Persist artifacts -----
np.save("embeddings_bge_E.npy", E)
group_df.to_parquet("groups.parquet")
faiss.write_index(index, "faiss_ip.index")

In [ ]:
K = 10
TOPK = min(K+1, N)
D_all, I_all = index.search(E, TOPK)

rows = []
texts = group_df["text"].tolist()
gids  = group_df["groupId"].to_numpy()

for i in range(N):
    cand_idx = [j for j in I_all[i] if j != i][:K]
    pairs = [[texts[i], texts[j]] for j in cand_idx]            # (src, rec) pairs
    ce_scores = rer.compute_score(pairs, batch_size=32)         # cross-encoder scores
    order = np.argsort(-np.asarray(ce_scores))                  # sort desc by CE score

    for rank, t in enumerate(order, 1):
        j = cand_idx[t]
        rows.append((gids[i], gids[j], float(D_all[i][np.where(I_all[i]==j)][0]),
                     float(ce_scores[t]), rank))

recs = pd.DataFrame(rows, columns=["src_groupId","rec_groupId","similarity","rerank_score","rec_rank"])


NameError: name 'N' is not defined

In [ ]:
# Find src_groupId that have less than three rec (i.e., zero recs)
src_counts = recs["src_groupId"].value_counts()
no_rec_srcs = src_counts[src_counts < 3]
print("src_groupId with less than one rec:", no_rec_srcs.index.tolist())


src_groupId with less than one rec: []


In [ ]:
recs[recs["src_groupId"] == "290281"] # reflex
#recs[recs["src_groupId"] == "200970"] # random
#recs[recs["src_groupId"] == "292706"] #dildo
#recs[recs["src_groupId"] == "291534"]

,src_groupId,rec_groupId,similarity,rec_rank,src_text,src_brand,src_categories_norm_full,src_colors_norm,rec_text,rec_brand,rec_categories_norm_full,rec_colors_norm
9150,290281,294785,0.649926,1,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],Reflex Snap On Reflector. Snap-on reflex som är enkel att fästa runt armar och ben. SpringYard Vardagshjälpmedel None.,SpringYard,[Vardagshjälpmedel],[]
9151,290281,290282,0.606378,2,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],Reflexbälte stretch. Reflexbälte så du syns bra ute i mörkret! Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[]
9152,290281,290299,0.598651,3,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],"Väska reflex. Tygkasse i reflextyg.Mått: 42x38x8 cm Väskor, Vardagshjälpmedel None.",unknown,"[Väskor, Vardagshjälpmedel]",[]
9153,290281,290182,0.556411,4,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],"Hopfällbar Krycka. Hopfällbar krycka som är mycket populär resekrycka, men är lika omtyckt för användning till vardags. Good Living Gånghjälpmedel None.",Good Living,[Gånghjälpmedel],[svart]
9154,290281,200231,0.552933,5,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],"Reflexmössa Herr. Varm och skön mössa. Åshild Accessoarer, Herr akryl, polyester.",Åshild,"[Accessoarer, Herr, Kepsar & mössor]",[svart]
9155,290281,281030,0.538784,6,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],Stövel. Pälsfodrad lättviksstövel som fungerar bra i kallt väder. Skor nylon.,unknown,[Skor],[sand]
9156,290281,290178,0.534825,7,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],"Rollator Let' Shop. Let&#039;s Shop hjälper dig att transportera dina föremål vart du än går - snabbt, enkelt och säkert.Denna lätta aluminium rullator är monterad med en stor shoppingväska som hjälper dig transportera Trust Care Gånghjälpmedel, Rollatorer None.",Trust Care,"[Gånghjälpmedel, Rollatorer]",[]
9157,290281,440447,0.526648,8,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],"Uppladdningsbar clip on Lampa. Denna mångsidiga och praktiska mini lampa erbjuder ett flexibelt, justerbart halsdesign och dimbar belysning för att passa dina behov. PURElite Synhjälpmedel, Vardagshjälpmedel None.",PURElite,"[Synhjälpmedel, Vardagshjälpmedel, Belysning]",[vit]
9158,290281,525050,0.524604,9,Reflex Pom Pom. Snygg och trendig reflex boll Pom-Pom med metallhake som gör att man kan lätt kan flytta reflexen från till exempel jackan till väskan. Vardagshjälpmedel None.,unknown,[Vardagshjälpmedel],[],"Natt & Garderobslampa. Praktiska LED-puckar som är perfekta till garderober, förråd och andra mörka utrymmen där man behöver en liten extra ljuspunkt. Star Trading Belysning, Barnrummet None.",Star Trading,"[Belysning, Barnrummet, Natt & garderobslampor]",[vit]
9159,290281,525051,0.5